In [1]:
import utils
import os
import keras
import numpy as np
import operator
import tensorflow as tf

from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [20]:
"""
extract_n_words = 1000000
# Convert the FastText pretrained vectors to binary format
# Binary format speed up model loading considerably
cvt_fname = 'wiki_1M'

vector_dir = 'vectors/wiki.fr'
fname = 'wiki.fr.vec'

utils.convert_vectors(vector_dir, fname, extract_n_words, cvt_fname)

vector_dir = 'vectors/wiki.zh'
fname = 'wiki.zh.vec'

utils.convert_vectors(vector_dir, fname, extract_n_words, cvt_fname)

vector_dir = 'vectors/wiki.en'
fname = 'wiki.en.vec'

utils.convert_vectors(vector_dir, fname, extract_n_words, cvt_fname)
"""

"\nextract_n_words = 1000000\n# Convert the FastText pretrained vectors to binary format\n# Binary format speed up model loading considerably\ncvt_fname = 'wiki_1M'\n\nvector_dir = 'vectors/wiki.fr'\nfname = 'wiki.fr.vec'\n\nutils.convert_vectors(vector_dir, fname, extract_n_words, cvt_fname)\n\nvector_dir = 'vectors/wiki.zh'\nfname = 'wiki.zh.vec'\n\nutils.convert_vectors(vector_dir, fname, extract_n_words, cvt_fname)\n\nvector_dir = 'vectors/wiki.en'\nfname = 'wiki.en.vec'\n\nutils.convert_vectors(vector_dir, fname, extract_n_words, cvt_fname)\n"

In [2]:
vector_dir = 'vectors/wiki.fr'
fr_model = utils.load_vectors(os.path.join(vector_dir, 'wiki_1M.fr.bin'), binary = True)

INFO:utils:Imported 1000000 words pretrained vector


In [3]:
vector_dir = 'vectors/wiki.en'
en_model = utils.load_vectors(os.path.join(vector_dir, 'wiki_1M.en.bin'), binary = True)

INFO:utils:Imported 1000000 words pretrained vector


In [144]:
nb_words = 10000
embedding_dim = 300
en_embedding, en_word2idx = utils.pretrained_embeddings('global-voices-2017q3.en', en_model, nb_words, embedding_dim)

INFO:utils:Number of words in the corpus: 367335
INFO:utils:Number of corpus words in the word2vec model: 143819


In [145]:
fr_embedding, fr_word2idx = utils.pretrained_embeddings('global-voices-2017q3.fr', fr_model, nb_words, embedding_dim)

INFO:utils:Number of words in the corpus: 217462
INFO:utils:Number of corpus words in the word2vec model: 111760


## GAN Architecture

In [146]:
# Model Inputs
def model_inputs(embedding_dim):
    source_embedding = tf.placeholder(tf.float32, (None, embedding_dim))
    target_embedding = tf.placeholder(tf.float32, (None, embedding_dim))
    return source_embedding, target_embedding

In [147]:
def generator(source_embedding, embedding_dim, reuse = False):
    with tf.variable_scope('generator', reuse=reuse):
        #embedding_dim = int(source_embedding.get_shape()[1])
        translation_mat = tf.Variable(initial_value=tf.truncated_normal(shape = (embedding_dim, embedding_dim), stddev = 0.05))
        translation = tf.matmul(source_embedding, translation_mat)

    return translation

In [148]:
# Discriminator
def discriminator(x, n_units = 2048, reuse = False, alpha = 0.01):
    with tf.variable_scope('discriminator', reuse = reuse):
        # First hidden unit
        h1 = tf.layers.dense(x, n_units, activation = None)
        # Leaky ReLU
        h1 = tf.maximum(h1 * alpha, h1)

        # Second hidden unit
        h2 = tf.layers.dense(h1, n_units, activation = None)
        # Leaky ReLU
        h2 = tf.maximum(h2 * alpha, h2)

        # Output
        logits = tf.layers.dense(h2, 1, activation = None)
        out = tf.sigmoid(logits)    
    return out, logits

In [149]:
# Leak factor for leaky ReLU
alpha = 0.01
# Smoothing 
smooth = 0.2

In [150]:
tf.reset_default_graph()
# Create our input placeholders
source_embedding, target_embedding = model_inputs(embedding_dim)

# Build the model
g_embedding = generator(source_embedding, embedding_dim)

d_model_real, d_logits_real = discriminator(target_embedding, n_units=2048, alpha=alpha)
d_model_fake, d_logits_fake = discriminator(g_embedding, reuse=True, n_units=2048, alpha=alpha)

In [151]:
# Calculate losses
d_loss_real = tf.reduce_mean(
                  tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_real, 
                                                          labels=tf.ones_like(d_logits_real) * (1 - smooth)))
d_loss_fake = tf.reduce_mean(
                  tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, 
                                                          labels=tf.zeros_like(d_logits_real)))
d_loss = d_loss_real + d_loss_fake

g_loss = tf.reduce_mean(
             tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake,
                                                     labels=tf.ones_like(d_logits_fake)))

In [152]:
# Optimizers
learning_rate = 0.002

# Get the trainable_variables, split into G and D parts
t_vars = tf.trainable_variables()
g_vars = [var for var in t_vars if var.name.startswith('generator')]
d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

d_train_opt = tf.train.AdamOptimizer(learning_rate).minimize(d_loss, var_list=d_vars)
g_train_opt = tf.train.AdamOptimizer(learning_rate).minimize(g_loss, var_list=g_vars)

In [158]:
batch_size = 100
epochs = 5
embedding_samples = []
losses = []
# Only save generator variables
saver = tf.train.Saver(var_list=g_vars)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for e in range(epochs):
        # create an index of the words to train
        sample = np.arange(0, nb_words, 1)
        np.random.shuffle(sample)
        sample = sample.reshape(-1, batch_size)
        for i in range(nb_words//batch_size):
            batch_idx = sample[i]
            batch_source = en_embedding[batch_idx]
            batch_target = fr_embedding[batch_idx]
            
            # Run optimizers
            _ = sess.run(d_train_opt, feed_dict={source_embedding: batch_source, target_embedding: batch_target})
            _ = sess.run(g_train_opt, feed_dict={source_embedding: batch_source})
        
        # At the end of each epoch, get the losses and print them out
        train_loss_d = sess.run(d_loss, {source_embedding: batch_source, target_embedding: batch_target})
        train_loss_g = g_loss.eval({source_embedding: batch_source})
            
        print("Epoch {}/{}...".format(e+1, epochs),
              "Discriminator Loss: {:.4f}...".format(train_loss_d),
              "Generator Loss: {:.4f}".format(train_loss_g))    
        # Save losses to view after training
        losses.append((train_loss_d, train_loss_g))
        
        # Sample from generator as we're training for viewing afterwards
        
        gen_embeddings = sess.run(g_embedding,
                                  feed_dict={source_embedding: en_embedding})
        embedding_samples.append(gen_embeddings)
        saver.save(sess, './checkpoints/generator.ckpt')

# Save training generator samples
#with open('train_samples.pkl', 'wb') as f:
#    pkl.dump(embedding_samples, f)

Epoch 1/5... Discriminator Loss: 0.6212... Generator Loss: 4.6360
Epoch 2/5... Discriminator Loss: 0.5204... Generator Loss: 38.9910
Epoch 3/5... Discriminator Loss: 0.5367... Generator Loss: 28.3164
Epoch 4/5... Discriminator Loss: 0.5016... Generator Loss: 13.5128
Epoch 5/5... Discriminator Loss: 0.5010... Generator Loss: 10.7074


In [159]:
def euclidean_dist_neighbor(x, y, k):
    dist = np.sum((x - y)**2, axis = 1)
    return np.argsort(dist)[:k]

In [160]:
fr_idx2word = {i: word for word, i in fr_word2idx.items()}

In [170]:
en_words = ['treaty', 'work']
fr_words = ['traite','travail']
for embedding in embedding_samples:
    #print('Iteration {}'.format(i+1))
    for word in en_words:
        idx = en_word2idx[word]
        gen_word_vec = embedding[idx]

        translation_idx = euclidean_dist_neighbor(gen_word_vec, fr_embedding, 3000)

        translations = [fr_idx2word[idx] for idx in translation_idx if fr_idx2word[idx] in fr_words]
        
        print('{}: {}'.format(word, translations))

treaty: ['traite']
work: ['traite', 'travail']
treaty: ['travail']
work: ['travail']
treaty: []
work: []
treaty: ['traite']
work: ['traite', 'travail']
treaty: []
work: []
